## Cleaning up tracks.csv

Clean up tracks.csv from the fma repo and save it as 'clean_tracks.csv'

In [2]:
%matplotlib inline

import os
import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
#Built-in module from FMA
import utils


In [5]:
filepath='C:\\Users\\ramir\\OneDrive\\Documents\\GitHub\\fma_metadata\\tracks.csv'
tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])


SUBSETS = ('small', 'medium', 'large')
tracks['set', 'subset'] = tracks['set', 'subset'].astype(
        'category', categories=SUBSETS, ordered=True)
    
### Basic columns for data analysis
basic_columns = [('album','date_released'), ('album', 'title'), ('album', 'id'),
                ('artist', 'id'), ('artist', 'name'),  
                 ('track', 'genres_top'), ('track', 'genres_all'),
                 ('track', 'title'), ('track', 'duration')]


### Set tracks df to just basic columns
tracks = tracks[basic_columns]

### Remove rows with blank genres
tracks = tracks[~tracks[('track','genres_all')].isnull()]

### Remove rows with genres that don't have genre codes, which are mislabeled. 
tracks.drop(tracks[tracks[('track', 'genres_all')].str.contains('[a-z]')].index, inplace=True)

### Replace blank top genres with 'UNKNOWN'
tracks[('track', 'genres_top')].fillna('UNKNOWN', inplace=True)

### Columns that should be in datetime format
COLUMNS_dt = [('album', 'date_released')]

### Assign datetime columns to datetime type
for column in COLUMNS_dt:    
    times = pd.to_datetime(tracks[column], errors='coerce')
    years = [time.year for time in times]
    tracks.loc[:,column] = years
    
    # Assume that any artist that doesn't have an active year end is still active
    if (column == ('artist', 'active_year_end')):
        
        tracks[column].fillna(pd.to_datetime('today').year, inplace=True)
        
    else:
        tracks[column].fillna(0, inplace=True)
        
### Columns that should be categories
COLUMNS_cat = [('track', 'genres_top')]

for column in COLUMNS_cat:
    tracks[column] = tracks[column].astype('category')
    
### Columns that should be numeric
COLUMNS_int = [('track', 'duration')]

for column in COLUMNS_int:
    tracks[column] = pd.to_numeric(tracks[column], errors='coerce')
    tracks[column].fillna(0, inplace=True)

### Columns that should be lists
COLUMNS_list = [('track', 'genres_all')]
for column in COLUMNS_list:
    tracks[column] = tracks[column].map(ast.literal_eval)
    
### Remove MultiIndexing
tracks.columns = tracks.columns.droplevel()
tracks.columns = ['year_released', 'album', 'album_id', 'artist_id', 'artist', 'genres_top', 'genres_all', 'track', 'duration']
    
tracks.head(10)


C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5,8,18,20,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead


year_released                 album album_id artist_id      artist  \
track_id                                                                       
2                2009.0  AWOL - A Way Of Life        1         1        AWOL   
3                2009.0  AWOL - A Way Of Life        1         1        AWOL   
5                2009.0  AWOL - A Way Of Life        1         1        AWOL   
10               2008.0     Constant Hitmaker        6         6   Kurt Vile   
20               2009.0                 Niris        4         4  Nicky Cook   
26               2009.0                 Niris        4         4  Nicky Cook   
30               2009.0                 Niris        4         4  Nicky Cook   
46               2009.0                 Niris        4         4  Nicky Cook   
48               2009.0                 Niris        4         4  Nicky Cook   
134              2009.0  AWOL - A Way Of Life        1         1        AWOL   

         genres_top         genres_all                track  duration  
track_id                                                               
2           Hip-Hop               [21]                 Food     168.0  
3           Hip-Hop               [21]         Electric Ave     237.0  
5           Hip-Hop               [21]           This World     206.0  
10              Pop               [10]              Freeway     161.0  
20          UNKNOWN  [17, 10, 76, 103]      Spiritual Level     311.0  
26          UNKNOWN  [17, 10, 76, 103]  Where is your Love?     181.0  
30          UNKNOWN  [17, 10, 76, 103]            Too Happy     174.0  
46          UNKNOWN  [17, 10, 76, 103]             Yosemite     104.0  
48          UNKNOWN  [17, 10, 76, 103]       Light of Light     205.0  
134         Hip-Hop               [21]         Street Music     207.0

In [6]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 106313 entries, 2 to 155320
Data columns (total 9 columns):
year_released    106313 non-null float64
album            105273 non-null object
album_id         106313 non-null object
artist_id        106270 non-null object
artist           106313 non-null object
genres_top       106313 non-null category
genres_all       106313 non-null object
track            106017 non-null object
duration         106313 non-null float64
dtypes: category(1), float64(2), object(6)
memory usage: 7.5+ MB


In [9]:
filepath='C:\\Users\\ramir\\OneDrive\\Documents\\GitHub\\fma_metadata\\tracks_clean.csv'
tracks.to_csv(filepath)

In [24]:
filepath='C:\\Users\\ramir\\OneDrive\\Documents\\GitHub\\fma_metadata\\tracks_clean.csv'
tracks_clean = pd.read_csv(filepath, index_col='track_id')
tracks_clean.head()

year_released                 album album_id artist_id      artist  \
track_id                                                                       
2                2009.0  AWOL - A Way Of Life        1         1        AWOL   
3                2009.0  AWOL - A Way Of Life        1         1        AWOL   
5                2009.0  AWOL - A Way Of Life        1         1        AWOL   
10               2008.0     Constant Hitmaker        6         6   Kurt Vile   
20               2009.0                 Niris        4         4  Nicky Cook   

         genres_top         genres_all            track  duration  
track_id                                                           
2           Hip-Hop               [21]             Food     168.0  
3           Hip-Hop               [21]     Electric Ave     237.0  
5           Hip-Hop               [21]       This World     206.0  
10              Pop               [10]          Freeway     161.0  
20          UNKNOWN  [17, 10, 76, 103]  Spiritual Level     311.0

In [11]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 106313 entries, 2 to 155320
Data columns (total 9 columns):
year_released    106313 non-null float64
album            105273 non-null object
album_id         106313 non-null object
artist_id        106270 non-null object
artist           106313 non-null object
genres_top       106313 non-null category
genres_all       106313 non-null object
track            106017 non-null object
duration         106313 non-null float64
dtypes: category(1), float64(2), object(6)
memory usage: 7.5+ MB


## Cleaning Up Echonest
### I: Extracting Basic Audio Features to echonest_audio_features.csv


In [3]:
filepath='C:\\Users\\ramir\\OneDrive\\Documents\\GitHub\\fma_metadata\\echonest.csv'
echonest = utils.load(filepath)
echonest.head()

echonest                                                    \
         audio_features                                                     
           acousticness danceability    energy instrumentalness  liveness   
track_id                                                                    
2              0.416675     0.675894  0.634476         0.010628  0.177647   
3              0.374408     0.528643  0.817461         0.001851  0.105880   
5              0.043567     0.745566  0.701470         0.000697  0.373143   
10             0.951670     0.658179  0.924525         0.965427  0.115474   
134            0.452217     0.513238  0.560410         0.019443  0.096567   

                                                                        \
                                           metadata                      
         speechiness    tempo   valence  album_date         album_name   
track_id                                                                 
2           0.159310  165.922  0.576661         NaN                NaN   
3           0.461818  126.957  0.269240         NaN                NaN   
5           0.124595  100.260  0.621661         NaN                NaN   
10          0.032985  111.562  0.963590  2008-03-11  Constant Hitmaker   
134         0.525519  114.290  0.894072         NaN                NaN   

             ...                                                       \
             ...     temporal_features                                  
             ...                   214        215       216       217   
track_id     ...                                                        
2            ...             -1.992303   6.805694  0.233070  0.192880   
3            ...             -1.582331   8.889308  0.258464  0.220905   
5            ...             -2.288358  11.527109  0.256821  0.237820   
10           ...             -3.662988  21.508228  0.283352  0.267070   
134          ...             -1.452696   2.356398  0.234686  0.199550   

                                                                        
                                                                        
               218      219       220       221        222         223  
track_id                                                                
2         0.027455  0.06408   3.67696   3.61288  13.316690  262.929749  
3         0.081368  0.06413   6.08277   6.01864  16.673548  325.581085  
5         0.060122  0.06014   5.92649   5.86635  16.013849  356.755737  
10        0.125704  0.08082   8.41401   8.33319  21.317064  483.403809  
134       0.149332  0.06440  11.26707  11.20267  26.454180  751.147705  

[5 rows x 249 columns]

In [7]:
echonest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13129 entries, 2 to 124911
Columns: 249 entries, (echonest, audio_features, acousticness) to (echonest, temporal_features, 223)
dtypes: float64(244), object(5)
memory usage: 25.0+ MB


In [8]:
echonest.columns

MultiIndex(levels=[['echonest'], ['audio_features', 'metadata', 'ranks', 'social_features', 'temporal_features'], ['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '12

In [14]:
echonest.columns = echonest.columns.droplevel()
#echonest['echonest', 'audio_features']

In [15]:
echonest.columns

MultiIndex(levels=[['audio_features', 'metadata', 'ranks', 'social_features', 'temporal_features'], ['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '12

In [17]:
echonest_audio_features = echonest['audio_features']

In [18]:
echonest_audio_features.head()

acousticness  danceability    energy  instrumentalness  liveness  \
track_id                                                                     
2             0.416675      0.675894  0.634476          0.010628  0.177647   
3             0.374408      0.528643  0.817461          0.001851  0.105880   
5             0.043567      0.745566  0.701470          0.000697  0.373143   
10            0.951670      0.658179  0.924525          0.965427  0.115474   
134           0.452217      0.513238  0.560410          0.019443  0.096567   

          speechiness    tempo   valence  
track_id                                  
2            0.159310  165.922  0.576661  
3            0.461818  126.957  0.269240  
5            0.124595  100.260  0.621661  
10           0.032985  111.562  0.963590  
134          0.525519  114.290  0.894072

In [19]:
echonest_audio_features.describe()

acousticness  danceability        energy  instrumentalness  \
count  1.312900e+04  13129.000000  13129.000000      13129.000000   
mean   5.246876e-01      0.487290      0.537516          0.640536   
std    3.837186e-01      0.190148      0.278049          0.361430   
min    9.035000e-07      0.051307      0.000020          0.000000   
25%    1.037726e-01      0.344759      0.321300          0.323466   
50%    5.739848e-01      0.485635      0.549113          0.838134   
75%    9.207270e-01      0.629094      0.776254          0.918244   
max    9.957965e-01      0.968645      0.999964          0.998016   

           liveness   speechiness         tempo       valence  
count  13129.000000  13129.000000  13129.000000  13129.000000  
mean       0.187804      0.099174    123.080061      0.439761  
std        0.158051      0.137381     35.015137      0.276028  
min        0.025297      0.022324     12.753000      0.000010  
25%        0.101406      0.036932     95.967000      0.197321  
50%        0.119002      0.049019    120.057000      0.417743  
75%        0.211041      0.085452    145.318000      0.665575  
max        0.980330      0.966177    251.072000      0.999990

In [20]:
echonest_audio_features.to_csv('C:\\Users\\ramir\\OneDrive\\Documents\\GitHub\\fma_metadata\\echonest_audio_features.csv')

In [22]:
filepath='C:\\Users\\ramir\\OneDrive\\Documents\\GitHub\\fma_metadata\\echonest_audio_features.csv'
echonest_audio_features_copy = pd.read_csv(filepath, index_col='track_id')
echonest_audio_features_copy.head()

acousticness  danceability    energy  instrumentalness  liveness  \
track_id                                                                     
2             0.416675      0.675894  0.634476          0.010628  0.177647   
3             0.374408      0.528643  0.817461          0.001851  0.105880   
5             0.043567      0.745566  0.701470          0.000697  0.373143   
10            0.951670      0.658179  0.924525          0.965427  0.115474   
134           0.452217      0.513238  0.560410          0.019443  0.096567   

          speechiness    tempo   valence  
track_id                                  
2            0.159310  165.922  0.576661  
3            0.461818  126.957  0.269240  
5            0.124595  100.260  0.621661  
10           0.032985  111.562  0.963590  
134          0.525519  114.290  0.894072

In [ ]:
t